In [18]:
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader

from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.postprocessor import SimilarityPostprocessor

import chromadb
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext


In [19]:
llm = Ollama(model="llama3.1:8b", request_timeout=1000)
Settings.llm = llm
embedding_model =   HuggingFaceEmbedding(model_name="BAAI/bge-small-en-v1.5")
Settings.embed_model = embedding_model

In [20]:
# Load data
documents = SimpleDirectoryReader("./doc/").load_data()

In [21]:
# Create vector store index with Chroma

# initialize client, setting path to save data
db = chromadb.PersistentClient(path="./ChromaDB")

# create collection
chroma_collection = db.get_or_create_collection("RagTest")

# assign chroma as the vector_store to the context
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
storage_context = StorageContext.from_defaults(vector_store=vector_store)

# create your index
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)


# load your index from stored vectors if already created 
#index = VectorStoreIndex.from_vector_store(vector_store, storage_context=storage_context)

# Create query engine

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10,
)

# configure response synthesizer
response_synthesizer = get_response_synthesizer()

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
    node_postprocessors=[SimilarityPostprocessor(similarity_cutoff=0.5)],
)

# query
response = query_engine.query("When world war 4 ended ?")
print("***********")
print(response)

***********
A fragile truce was signed in Geneva-2 in year 2010.
